In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import random
from sklearn.model_selection import train_test_split
import wandb

# Set random seeds for reproducibility
random_state = 42
np.random.seed(random_state)
random.seed(random_state)
torch.manual_seed(random_state)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(random_state)

# ----- Load Stock and Sentiment Data -----
stock_data = pd.read_csv('AAPL_trend.csv', parse_dates=['date'])
stock_data = stock_data[stock_data['trend'] != 'stable']
stock_data['closingValue'] = stock_data['closingValue'].astype(float)
sentiment_data = pd.read_csv('sentiment_scores_title.csv', parse_dates=['date'])
stock_data['date'] = pd.to_datetime(stock_data['date'])
merged_data = pd.merge(stock_data, sentiment_data, on='date', how='left')

# ----- Define Date Range -----
start_date = pd.to_datetime("2024-02-12")
end_date = pd.to_datetime("2025-02-19")
selected_stock_data = merged_data[(merged_data['date'] >= start_date) & (merged_data['date'] <= end_date)]

# ----- Feature Engineering -----
X_days = 7
Y_days = 1
selected_stock_data = selected_stock_data.sort_values('date')
closing_values = selected_stock_data['closingValue'].values
sentiment_scores = selected_stock_data['sentiment_score'].values
trends = selected_stock_data['trend'].values

# Map trends to numerical labels
trend_to_label = {'increase': 0, 'decrease': 1, 'stable': 0}
labels_all = [trend_to_label[t] for t in trends]

features, labels = [], []
for i in range(max(X_days, Y_days), len(selected_stock_data)):
    stock_features = closing_values[i - X_days:i]
    sentiment_feature = sentiment_scores[i - Y_days:i]
    feature_vector = np.concatenate([stock_features, sentiment_feature])
    features.append(feature_vector)
    labels.append(labels_all[i])

features = np.array(features)
labels = np.array(labels)

# Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(
    features, labels, test_size=0.2, random_state=random_state, stratify=labels
)

# Define LSTM Model
class StockTrendLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(StockTrendLSTM, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)
    
    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size)
        out, _ = self.lstm(x, (h0, c0))
        out = out[:, -1, :]
        out = self.fc(out)
        return out

# Prepare data for LSTM
input_size = 1
X_train_seq = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test_seq = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)
X_train_tensor = torch.tensor(X_train_seq, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test_seq, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

# ----- Define W&B Sweep Configuration -----
sweep_config = {
    "method": "bayes",  # Can also be "random" or "grid"
    "metric": {"name": "accuracy", "goal": "maximize"},
    "parameters": {
        "hidden_size": {"values": [16, 32, 64]},
        "num_layers": {"values": [1, 2, 3]},
        "lr": {"values": [0.01, 0.001, 0.0001]},
        "num_epochs": {"value": 10000}  # Keep fixed if not tuning
    }
}

sweep_id = wandb.sweep(sweep_config, project="stock_trend_lstm")


# ----- Training Function -----
def train():
    wandb.init()
    config = wandb.config  # Load sweep parameters

    model = StockTrendLSTM(
        input_size=1,
        hidden_size=config.hidden_size,
        num_layers=config.num_layers,
        num_classes=3
    )
    
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=config.lr)

    num_epochs = config.num_epochs
    for epoch in range(num_epochs):
        model.train()
        outputs = model(X_train_tensor)
        loss = criterion(outputs, y_train_tensor)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (epoch + 1) % 100 == 0:
            model.eval()
            with torch.no_grad():
                test_outputs = model(X_test_tensor)
                _, predicted = torch.max(test_outputs, 1)
                accuracy = (predicted == y_test_tensor).sum().item() / y_test_tensor.size(0)

                # Log hyperparameters & metrics
                wandb.log({
                    "epoch": epoch + 1,
                    "loss": loss.item(),
                    "accuracy": accuracy,
                    "hidden_size": config.hidden_size,
                    "num_layers": config.num_layers,
                    "lr": config.lr
                })

                print(f"Epoch {epoch + 1}: Accuracy = {accuracy:.4f}")

    wandb.finish()

# ----- Step 3: Run Sweep -----
wandb.agent(sweep_id, train, count=10)  # Run 10 different configurations


wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:wandb: Paste an API key from your profile and hit enter:wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: C:\Users\sanni\_netrc


Create sweep with ID: pgem3zdy
Sweep URL: https://wandb.ai/its_mrpsycho/stock_trend_lstm/sweeps/pgem3zdy


wandb: Agent Starting Run: f50aadaq with config:
wandb: 	hidden_size: 16
wandb: 	lr: 0.001
wandb: 	num_epochs: 10000
wandb: 	num_layers: 2
wandb: Currently logged in as: its_mrpsycho to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch 100: Accuracy = 0.5800
Epoch 200: Accuracy = 0.5800
Epoch 300: Accuracy = 0.5800
Epoch 400: Accuracy = 0.5800
Epoch 500: Accuracy = 0.5800
Epoch 600: Accuracy = 0.5800
Epoch 700: Accuracy = 0.5800
Epoch 800: Accuracy = 0.5800
Epoch 900: Accuracy = 0.5800
Epoch 1000: Accuracy = 0.5800
Epoch 1100: Accuracy = 0.5800
Epoch 1200: Accuracy = 0.5800
Epoch 1300: Accuracy = 0.5800
Epoch 1400: Accuracy = 0.5800
Epoch 1500: Accuracy = 0.5800
Epoch 1600: Accuracy = 0.5800
Epoch 1700: Accuracy = 0.5800
Epoch 1800: Accuracy = 0.5800
Epoch 1900: Accuracy = 0.5800
Epoch 2000: Accuracy = 0.5800
Epoch 2100: Accuracy = 0.5800
Epoch 2200: Accuracy = 0.5800
Epoch 2300: Accuracy = 0.5800
Epoch 2400: Accuracy = 0.5800
Epoch 2500: Accuracy = 0.5400
Epoch 2600: Accuracy = 0.5800
Epoch 2700: Accuracy = 0.5600
Epoch 2800: Accuracy = 0.5400
Epoch 2900: Accuracy = 0.5800
Epoch 3000: Accuracy = 0.5800
Epoch 3100: Accuracy = 0.5400
Epoch 3200: Accuracy = 0.5400
Epoch 3300: Accuracy = 0.5400
Epoch 3400: Accurac

accuracy,▆▆▆▆▆▆▅▅▆▆▅▅▇▅▆▇▇▅█▇▁▇▇▇▆▆▅▇▇▇▇▇▇▃▅▅▄▅▃▄
epoch,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▄▄▄▄▅▅▅▅▆▆▆▆▆▇▇▇▇▇█
hidden_size,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
loss,▇▇▇▇▇▇▇▇▇▇▇▇▆▆▅▅▅▅▄▄▂▂▆▃▂▄▂▁▂▁▂▁▁▁▁▁▁▁█▁
lr,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
num_layers,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
accuracy,0.52
epoch,10000
hidden_size,16
loss,0.50728
lr,0.001


...

Epoch 100: Accuracy = 0.5800
Epoch 200: Accuracy = 0.5800
Epoch 300: Accuracy = 0.5800
Epoch 400: Accuracy = 0.5800
Epoch 500: Accuracy = 0.5800
Epoch 600: Accuracy = 0.5800
Epoch 700: Accuracy = 0.5800
Epoch 800: Accuracy = 0.5800
Epoch 900: Accuracy = 0.5800
Epoch 1000: Accuracy = 0.5800
Epoch 1100: Accuracy = 0.5800
Epoch 1200: Accuracy = 0.5800
Epoch 1300: Accuracy = 0.5800
Epoch 1400: Accuracy = 0.5800
Epoch 1500: Accuracy = 0.5600
Epoch 1600: Accuracy = 0.5800
Epoch 1700: Accuracy = 0.6000
Epoch 1800: Accuracy = 0.5800
Epoch 1900: Accuracy = 0.5200
Epoch 2000: Accuracy = 0.5200
Epoch 2100: Accuracy = 0.5200
Epoch 2200: Accuracy = 0.5200
Epoch 2300: Accuracy = 0.5200
Epoch 2400: Accuracy = 0.5200
Epoch 2500: Accuracy = 0.5200
Epoch 2600: Accuracy = 0.5200
Epoch 2700: Accuracy = 0.5200
Epoch 2800: Accuracy = 0.5000
Epoch 2900: Accuracy = 0.5200
Epoch 3000: Accuracy = 0.5200
Epoch 3100: Accuracy = 0.5200
Epoch 3200: Accuracy = 0.5200
Epoch 3300: Accuracy = 0.5200
Epoch 3400: Accurac

accuracy,██████████▅▅▅▄▅▅▅▄▅▄▅▆▃▄▆▅▅▅▆▅▅▅▅▁▂▆▇▄▆▇
epoch,▁▁▂▂▂▂▂▂▃▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▇▇▇▇▇▇▇▇██
hidden_size,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
loss,█████▇▇▇▇▇▆▆▆▅▅▄▄▄▄▄▃▃▃▅▄▄▄▃▃▃▂▂▂▂▂▂▂▁▁▁
lr,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
num_layers,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
accuracy,0.54
epoch,10000
hidden_size,64
loss,0.51983
lr,0.0001


wandb: Agent Starting Run: 6xoihq3p with config:
wandb: 	hidden_size: 64
wandb: 	lr: 0.001
wandb: 	num_epochs: 10000
wandb: 	num_layers: 2


Epoch 100: Accuracy = 0.5800
Epoch 200: Accuracy = 0.5800
Epoch 300: Accuracy = 0.5800
Epoch 400: Accuracy = 0.5200
Epoch 500: Accuracy = 0.5400
Epoch 600: Accuracy = 0.5600
Epoch 700: Accuracy = 0.5400
Epoch 800: Accuracy = 0.5200
Epoch 900: Accuracy = 0.5600
Epoch 1000: Accuracy = 0.5400
Epoch 1100: Accuracy = 0.4600
Epoch 1200: Accuracy = 0.5600
Epoch 1300: Accuracy = 0.5400
Epoch 1400: Accuracy = 0.5400
Epoch 1500: Accuracy = 0.5600
Epoch 1600: Accuracy = 0.5600
Epoch 1700: Accuracy = 0.5400
Epoch 1800: Accuracy = 0.5600
Epoch 1900: Accuracy = 0.5600
Epoch 2000: Accuracy = 0.5400
Epoch 2100: Accuracy = 0.5800
Epoch 2200: Accuracy = 0.5200
Epoch 2300: Accuracy = 0.5600
Epoch 2400: Accuracy = 0.5800
Epoch 2500: Accuracy = 0.5200
Epoch 2600: Accuracy = 0.5400
Epoch 2700: Accuracy = 0.5400
Epoch 2800: Accuracy = 0.5200
Epoch 2900: Accuracy = 0.5600
Epoch 3000: Accuracy = 0.5600
Epoch 3100: Accuracy = 0.5600
Epoch 3200: Accuracy = 0.5800
Epoch 3300: Accuracy = 0.5600
Epoch 3400: Accurac

accuracy,▃▄▃▃▁▃▃▄▄▄▄▄▅▄▅▄▇▄▅▄▄▅▅▄▅▃▃▃▄▁▂█▆▆▆▄▇▅▅▆
epoch,▁▁▁▁▁▂▂▂▂▂▂▂▃▃▃▃▃▃▃▄▅▅▅▅▅▆▆▆▆▇▇▇▇▇▇▇████
hidden_size,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
loss,███▇▇▇▇▇▇▇▇▆▆▆▆▆▅▅▅▅▅▄▄▅▃█▃▃▂▂▂▂▁▁▁█▂▂▂▁
lr,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
num_layers,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
accuracy,0.64
epoch,10000
hidden_size,64
loss,0.115
lr,0.001
